In [2]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()
with open("../login.json", "r") as j:
    misDatos = json.load(j)
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [3]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id,CreatedDate from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1

100%|██████████| 1/1 [00:12<00:00, 12.78s/it]


In [4]:
df2.head()


,P_liza__c,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,CreatedDate,LLave,dato
0,Sin Póliza Asociada,Colaboradores Care Assistance Chile SPA,Colaboradores Care Assistance Chile SPA,775173998,Colaboradores,Siempre Chile SPA,Plan Gold,Chile,False,a3yRN0000006mmjYAA,1703777860000,775173998Sin Póliza Asociada,1
1,-,Apunto Servicios De Alimentacion,Apunto Servicios De Alimentacion,-,Consalud,Holding Consalud,-,Chile,False,a3yRN0000006rG9YAI,1703860945000,-,1
2,-,2M Inmobiliaria Spa.,2M Inmobiliaria Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGAYAY,1703860945000,-,1
3,-,360 Gms Capacitacion Spa.,360 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGBYAY,1703860945000,-,1
4,-,361 Gms Capacitacion Spa.,361 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGCYAY,1703860945000,-,1


In [5]:
df2.sort_values("CreatedDate", inplace=True)

In [6]:
df2.head()

,P_liza__c,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id,CreatedDate,LLave,dato
0,Sin Póliza Asociada,Colaboradores Care Assistance Chile SPA,Colaboradores Care Assistance Chile SPA,775173998,Colaboradores,Siempre Chile SPA,Plan Gold,Chile,False,a3yRN0000006mmjYAA,1703777860000,775173998Sin Póliza Asociada,1
93,-,Ag.De Aduanas Alejandro Etcheverry Arentsen Y ...,Ag.De Aduanas Alejandro Etcheverry Arentsen Y ...,-,Confuturo,Holding Contufuturo,-,Chile,False,a3yRN0000006rHdYAI,1703860945000,-,1
94,-,Agencia De Aduanas Carlos De Aguirre Y Cia Ltda.,Agencia De Aduanas Carlos De Aguirre Y Cia Ltda.,-,Confuturo,Holding Contufuturo,-,Chile,False,a3yRN0000006rHeYAI,1703860945000,-,1
95,-,Agencia De Aduanas Felipe Serrano Solar Y Comp...,Agencia De Aduanas Felipe Serrano Solar Y Comp...,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rHfYAI,1703860945000,-,1
96,-,Agencia De Viajes Mundo Tour Ltda.,Agencia De Viajes Mundo Tour Ltda.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rHgYAI,1703860945000,-,1


In [7]:
df3 = pd.DataFrame()

df3 = df2[[ "Id","Empresa_Padre__r.Name"]]
df3 = df3.drop_duplicates(
    [
        "Empresa_Padre__r.Name",
    ],
    keep="first",
)
df3["contar"] = 1



In [8]:
df3.head()


,Id,Empresa_Padre__r.Name,contar
0,a3yRN0000006mmjYAA,Colaboradores,1
93,a3yRN0000006rHdYAI,Confuturo,1
95,a3yRN0000006rHfYAI,THB Seguros COVID,1
98,a3yRN0000006rHiYAI,Lockton,1
91,a3yRN0000006rHbYAI,Consalud,1


In [9]:
df2 = pd.merge(
    left=df2,
    right=df3,
    how="left",
    left_on="Id",
    right_on="Id",
)

df2.contar.fillna(0, inplace=True)

In [10]:
df2.head()

,P_liza__c,Nombre_de_Empresa__c,Identificador__c,Rut_Empresa__c,Empresa_Padre__r.Name_x,Holding__c,Plan__c,Pais__c,Activo__c,Id,CreatedDate,LLave,dato,Empresa_Padre__r.Name_y,contar
0,Sin Póliza Asociada,Colaboradores Care Assistance Chile SPA,Colaboradores Care Assistance Chile SPA,775173998,Colaboradores,Siempre Chile SPA,Plan Gold,Chile,False,a3yRN0000006mmjYAA,1703777860000,775173998Sin Póliza Asociada,1,Colaboradores,1.0
1,-,Ag.De Aduanas Alejandro Etcheverry Arentsen Y ...,Ag.De Aduanas Alejandro Etcheverry Arentsen Y ...,-,Confuturo,Holding Contufuturo,-,Chile,False,a3yRN0000006rHdYAI,1703860945000,-,1,Confuturo,1.0
2,-,Agencia De Aduanas Carlos De Aguirre Y Cia Ltda.,Agencia De Aduanas Carlos De Aguirre Y Cia Ltda.,-,Confuturo,Holding Contufuturo,-,Chile,False,a3yRN0000006rHeYAI,1703860945000,-,1,NaN,0.0
3,-,Agencia De Aduanas Felipe Serrano Solar Y Comp...,Agencia De Aduanas Felipe Serrano Solar Y Comp...,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rHfYAI,1703860945000,-,1,THB Seguros COVID,1.0
4,-,Agencia De Viajes Mundo Tour Ltda.,Agencia De Viajes Mundo Tour Ltda.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rHgYAI,1703860945000,-,1,NaN,0.0


In [11]:
""" def contarsi(dato, cantidad_de_filas):
     for i in range(cantidad_de_filas):
        print(i) 
    for index, value in df['Nombre'].iteritems():
    print(f'En el índice {index}, el valor es {value}.')    """ 

SyntaxError: invalid syntax (3837862790.py, line 2)

In [ ]:
""" cantidad = df2.shape[0]

df2["conteo"] = df2["Empresa_Padre__r.Name"].apply(lambda y: contarsi(y, cantidad)) """

In [24]:
nuevos = "C:/Users/jmartinez/Documents/reportes_metlife2.xlsx"
Pacientes_Archivo = pd.read_excel(nuevos)
Pacientes_Archivo.head(5)

,Codigo Ficha,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza_x,Fecha,Fecha_chilena,...,Meses,Canal,Clínica,Motivo de consulta,Nombre Empresa,Corredor,Pais,Rut_Titular_y,Relación_y,Index
0,FSA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,Noviembre,LLamada,Orientación Médica,NaN,Prueba MetLife,NaN,Chile,169397759,Titular,1
1,SA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,Noviembre,Agendamiento,Orientación Médica,NaN,Prueba MetLife,NaN,Chile,169397759,Titular,2
2,FSA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,Noviembre,LLamada,Deporte,NaN,Clínica Alemana,Willis Towers Watson S.A. Corred. De Seg.,Chile,106879508,Titular,3
3,SA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,Noviembre,Agendamiento,Orientación Médica,NaN,Clínica Alemana,Willis Towers Watson S.A. Corred. De Seg.,Chile,106879508,Titular,4
4,FSA-0059,248270454,Aaron Antonio Aguayo Perez,Hombre,MetLife,Alsea,Starbucks Coffee Chile S.A.,340020238,2022-11-21T16:00:00.000+0000,2022-11-21 16:00:00,...,Noviembre,LLamada,Deporte,NaN,Starbucks Coffee Chile S.A.,Aon Risk Services (Chile) Corred. De Seg.,Chile,191841344,Carga,5


In [29]:
df3 = pd.DataFrame()
Pacientes_Archivo.sort_values("Fecha_chilena", inplace=True)
Pacientes_Archivo.Rut_Titular_y.fillna(Pacientes_Archivo["Rut"], inplace=True)
Pacientes_Archivo.head(5)
Pacientes_Archivo["llave"] = Pacientes_Archivo["Año"].astype(str) + Pacientes_Archivo["Rut_Titular_y"].astype(str)
df3 = Pacientes_Archivo[[ "Index","llave"]]
df3 = df3.drop_duplicates(
    [
        "llave",
    ],
    keep="first",
)
df3["contar"] = 1 



In [30]:
df3.head()

,Index,llave,contar
0,1,2022169397759,1
2,3,2022106879508,1
4,5,2022191841344,1
6,7,2022166636647,1
8,9,2022176041080,1


In [31]:
Pacientes_Archivo = pd.merge(
    left=Pacientes_Archivo,
    right=df3,
    how="left",
    left_on="Index",
    right_on="Index",
)

Pacientes_Archivo.contar.fillna(0, inplace=True)

In [32]:
Pacientes_Archivo.head(5)


,Codigo Ficha,Rut,Nombre,Sexo,Campaña,Holding,Empleador,Poliza_x,Fecha,Fecha_chilena,...,Motivo de consulta,Nombre Empresa,Corredor,Pais,Rut_Titular_y,Relación_y,Index,llave_x,llave_y,contar
0,FSA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,NaN,Prueba MetLife,NaN,Chile,169397759,Titular,1,2022169397759,2022169397759,1.0
1,SA-22078,169397759,Rodrigo González,Hombre,MetLife,Sin Holding Asociado,Prueba MetLife,Sin Póliza Asociada,2022-11-05T01:00:00.000+0000,2022-11-05 01:00:00,...,NaN,Prueba MetLife,NaN,Chile,169397759,Titular,2,2022169397759,NaN,0.0
2,FSA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,NaN,Clínica Alemana,Willis Towers Watson S.A. Corred. De Seg.,Chile,106879508,Titular,3,2022106879508,2022106879508,1.0
3,SA-0045,106879508,Leal Miranda Digna,Mujer,MetLife,Clínica Alemana,Clínica Alemana,340006119,2022-11-21T07:00:00.000+0000,2022-11-21 07:00:00,...,NaN,Clínica Alemana,Willis Towers Watson S.A. Corred. De Seg.,Chile,106879508,Titular,4,2022106879508,NaN,0.0
4,FSA-0059,248270454,Aaron Antonio Aguayo Perez,Hombre,MetLife,Alsea,Starbucks Coffee Chile S.A.,340020238,2022-11-21T16:00:00.000+0000,2022-11-21 16:00:00,...,NaN,Starbucks Coffee Chile S.A.,Aon Risk Services (Chile) Corred. De Seg.,Chile,191841344,Carga,5,2022191841344,2022191841344,1.0


In [33]:
from pivottablejs import pivot_ui

In [34]:
pivot_ui(Pacientes_Archivo)